In [1]:
%pip install git+https://github.com/bigscience-workshop/petals

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/bigscience-workshop/petals to /tmp/pip-req-build-zzg9wg8v
  Running command git clone --filter=blob:none --quiet https://github.com/bigscience-workshop/petals /tmp/pip-req-build-zzg9wg8v
  Resolved https://github.com/bigscience-workshop/petals to commit ae19b650959fadd837f4db799f9ec35011199506
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for bitsandbytes==0.41.1 from https://files.pythonhosted.org/packages/1e/2c/af22cd797fc368a9f098ed03015730e6568b884fe67f9940793d944a4b7b/bitsandbytes-0.41.1-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate>=0.22.0 from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/ac

  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/fe/d3/e1aa96437d944fbb9cc95d0316e25583886e9cd9e6adc07baad943524eda/fsspec-2023.9.2-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0.0,>=0.11.1 from https://files.pythonhosted.org/packages/aa/f3/3fc97336a0e90516901befd4f500f08d691034d387406fdbde85bea827cc/huggingface_hub-0.17.3-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.12.2 from https://files.pythonhosted.org/packages/c8/2c/03046cac73f46bfe98fc846ef629cf4f84c2f59258216aa2cc0d22bfca8f/protobuf-4.24.4-cp37-abi3-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for grpcio>=1.59.0 from https://files.pythonhosted.org/packages/20/7f/e76618521aa9d33c6c1c9c3473f866da521678aa6ea2f4df3a896757748c/grpcio-1.59.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for net

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 6.7 MB/s eta 0:00:006.5 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 4.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.6 MB/s eta 0:00:0031m21.0 MB/s eta 0:00:01
  Created wheel for petals: filename=petals-2.2.0-py3-none-any.whl size=114221 sha256=f5a2b1c401027f9b0ec42454ae5cf5c2c00ef4df6ce692ce27826198b8b0cf23
  Stored in directory: /tmp/pip-ephem-wheel-cache-q86ixw4i/wheels/98/38/8f/58b4f2d1d3762b120b9747915042f7498efd33a7d7475228ce
  Created wheel for hivemind: filename=hivemind-1.1.10.post2-py3-none-any.whl size=9011440 sha256=f40e62b5c713ff958fe2dd743b05f6bb99c3917fd80b8c9cc277be4a039822c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-q86ixw4i/wheels/53/98/e3/d51ae54ae8302c5a1872fae2e85a8e8291651af19f550965c1
  Created wheel for prefetch-generator: filename=prefetch_generator-1.0.3-py3-none-any.whl size=4758 sha256=6e3ffee4d0375063649bd46e50c43e1

In [3]:
import torch
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM

model_name = "petals-team/StableBeluga2"
# You could also use "meta-llama/Llama-2-70b-chat-hf" or any other supported model from 🤗 Model Hub

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, add_bos_token=False)
model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
model = model.cuda()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [ ]:
inputs = tokenizer('In the sentence "Recent work demonstrates the potential of multilingual pretraining of creating one model that can be used for various tasks in different languages. Previous work in multilingual pretraining has demonstrated that machine translation systems can be created by finetuning on bitext", multilingual means: "', return_tensors="pt")["input_ids"].cuda()
outputs = model.generate(inputs, max_new_tokens=3)
print(tokenizer.decode(outputs[0]))

In [ ]:
fake_token = tokenizer("^")["input_ids"][0]  # Workaround to make tokenizer.decode() keep leading spaces

text = "What is a good chatbot? Answer:"
prefix = tokenizer(text, return_tensors="pt")["input_ids"].cuda()

with model.inference_session(max_length=30) as sess:
    for i in range(20):
        # Prefix is passed only for the 1st token of the outputs
        inputs = prefix if i == 0 else None

        # Let's use sampling with temperature = 0.9 and top_p = 0.6 to get more diverse results
        outputs = model.generate(inputs, max_new_tokens=1, session=sess,
                                 do_sample=True, temperature=0.9, top_p=0.6)

        text += tokenizer.decode([fake_token, outputs[0, -1].item()])[1:]
        print(text)

In [ ]:
with model.inference_session(max_length=512) as sess:
    while True:
        prompt = input('Human: ')
        if prompt == "":
            break
        prefix = f"Human: {prompt}\nFriendly AI:"
        prefix = tokenizer(prefix, return_tensors="pt")["input_ids"].cuda()
        print("Friendly AI:", end="", flush=True)

        while True:
            outputs = model.generate(prefix, max_new_tokens=1, session=sess,
                                     do_sample=True, temperature=0.9, top_p=0.6)
            outputs = tokenizer.decode([fake_token, outputs[0, -1].item()])[1:]

            # Now, let's print one new token at a time
            print(outputs, end="", flush=True)

            if "\n" in outputs or "</s>" in outputs:
                break
            prefix = None  # Prefix is passed only for the 1st token of the bot's response